In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from biosppy.signals import eeg
from biosppy.signals import emg
import time


In [4]:
start = time.time()

# import train sets
train_eeg1_raw = pd.read_csv('files/train_eeg1.csv').drop('Id', axis=1).values[:21600]
train_eeg2_raw = pd.read_csv('files/train_eeg2.csv').drop('Id', axis=1).values[:21600]
train_emg_raw = pd.read_csv('files/train_emg.csv').drop('Id', axis=1).values[:21600]

# import test sets
test_eeg1_raw = pd.read_csv('files/test_eeg1.csv').drop('Id', axis=1).values[:21600]
test_eeg2_raw = pd.read_csv('files/test_eeg2.csv').drop('Id', axis=1).values[:21600]
test_emg_raw = pd.read_csv('files/test_emg.csv').drop('Id', axis=1).values[:21600]

# import labels
train_labels_raw = pd.read_csv('files/train_labels.csv').drop('Id', axis=1).values

print(train_eeg1_raw.shape, train_eeg2_raw.shape, train_emg_raw.shape)
print(test_eeg1_raw.shape, test_eeg2_raw.shape, test_emg_raw.shape)
print(train_labels_raw.shape)

print("Time: ", time.time() - start)

(21600, 512) (21600, 512) (21600, 512)
(21600, 512) (21600, 512) (21600, 512)
(64800, 1)
Time:  26.52317190170288


In [ ]:
output = pd.read_csv('files/sample.csv')
        for i in range(output.shape[0]):
            output.iat[i, 1] = y_predict[i]
        output.to_csv(f"outputs/{OvRClassifier.__class__.__name__}.{classifier.__class__.__name__}.csv", index=False)